# SMT

In [167]:
from itertools import combinations
from z3 import *

In [168]:
#Caricamento dati

file = open("./Instances/inst02.dat", "r")
lines = file.readlines()
file.close()
m = int(lines[0].rstrip(''))
n = int(lines[1].rstrip(''))
load = [int(x) for x in lines[2].split(' ')]
size = [int(x) for x in lines[3].split(' ')]
D = [[int(x) for x in line.rstrip().split(' ') ]for line in lines[4:]]
o = n
print(m)
print(n)
print(load)
print(size)
for i in range(n+1):
    for j in range(n+1):
        print(D[i][j], end = " ")
    print()

#Definizione solver
s = Solver()
# s = Optimize()

6
9
[190, 185, 185, 190, 195, 185]
[11, 11, 23, 16, 2, 1, 24, 14, 20]
0 199 119 28 179 77 145 61 123 87 
199 0 81 206 38 122 55 138 76 113 
119 81 0 126 69 121 26 117 91 32 
28 206 126 0 186 84 152 68 130 94 
169 38 79 176 0 92 58 108 46 98 
77 122 121 84 102 0 100 16 46 96 
145 55 26 152 58 100 0 91 70 58 
61 138 113 68 118 16 91 0 62 87 
123 76 91 130 56 46 70 62 0 66 
87 113 32 94 94 96 58 87 66 0 


In [169]:
#Utils
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars))

def all_false(bool_vars):
    return And([And(Not(pair[0]), Not(pair[1])) for pair in combinations(bool_vars, 2)])

In [170]:
#Decision variables
assignment = [[Bool(f'assignment_{i}_{j}') for j in range(m)] for i in range(n)]

# tour[i][j] = x ---> courier i dispatch item x at j-th route
tour = [[Int(f'tour_{i}_{j}') for j in range(n+2)] for i in range(m)]

In [171]:
#Constraints

# Each item is assigned to exactly one courier
for i in range(n):
    s.add(exactly_one_np([assignment[i][j] for j in range(m)]))

# Each courier cannot load more than his capacity
for j in range(m):
    s.add(sum([If(assignment[i][j], size[i], 0) for i in range(n)]) <= load[j])

# Initial and final destination are the same
for c in range(m):
    s.add(And(tour[c][0] == o, tour[c][n+1] == o))

# Assigned items must be in courier's tour
for c in range(m):
    for i in range(n):
        s.add(Implies(assignment[i][c], at_least_one_np([tour[c][j] == i for j in range(1, n+1)])))

# Each assigned item is dispatched just once
for c in range(m):
    for i in range(n):
        s.add(Implies(assignment[i][c], exactly_one_np([tour[c][j] == i for j in range(1,n+1)])))

# If courer c does not dispatch item i, then i is not in his tour
for c in range(m):
    for i in range(n):
        s.add(Implies(Not(assignment[i][c]), all_false([tour[c][j] == i for j in range(1, n+1)])))

# Can't go back to deposit
for c in range(m):
    for j in range(1,n):
        s.add(Implies(tour[c][j] == o, tour[c][j+1] == o))

# Having an item in the tour implies that the item is assigned
for c in range(m):
    for j in range(1,n+1):
        s.add([Implies(tour[c][j] == i, assignment[i][c]) for i in range(n)])

# Limit the range of assignable item to only valid one
for c in range(m):
    for j in range(1,n+1):
        s.add(And(Not(tour[c][j]>n), Not(tour[c][j]<0)))

In [190]:
if s.check() == sat:
    model = s.model()
    print("sat")
    for i in range(n):
        for j in range(m):
            print(model[assignment[i][j]], end = " ")
        print()
    for i in range(m):
        for j in range(n+2):
            print(model[tour[i][j]], end = " ")
        print()
    
    dist = [sum([ D[int(str(model[tour[i][j]]))][int(str(model[tour[i][j+1]]))] for j in range(n+1) if i in range(n)]) for i in range(m)]
    print("Distances:", dist)
    print("MaxDist:", max(dist))
    print("Total Dist:", sum(dist))
else:
    print("unsat")

sat
False True False False False False 
False False False False True False 
False True False False False False 
True False False False False False 
False False False False True False 
True False False False False False 
False False False False False True 
True False False False False False 
False False False False True False 
9 5 3 7 9 9 9 9 9 9 9 
9 0 2 9 9 9 9 9 9 9 9 
9 9 9 9 9 9 9 9 9 9 9 
9 9 9 9 9 9 9 9 9 9 9 
9 8 4 1 9 9 9 9 9 9 9 
9 6 9 9 9 9 9 9 9 9 9 
Distances [335, 238, 0, 0, 273, 116]
MaxDist 335
Total Dist 962
